# 03 – Finaler Durchlauf: beam=12, len=20 über alle Sessions (vor Bugfix)

## Kontext

Die Experimente in `02_`–`02h_` wurden auf einem **5-Session-Dev-Subset** durchgeführt.
Dieses Notebook führt den bisher besten Parameter-Stand (beam=12, max_length=20)
auf **allen Dev-Sessions** aus, um eine robuste Gesamtschätzung zu erhalten.

**Hinweis:** Dieser Lauf verwendet noch die Daten **vor dem Segmentierungs-Bugfix**. Der Bugfix-Lauf folgt in `03b_`.

## Ablauf

1. Inference auf allen Sessions mit `inference.main()` (auskommentiert – läuft nicht erneut)
2. Evaluation via `evaluate.py` als Subprocess
3. Regex-Parsing der Evaluations-Ausgabe → DataFrame
4. Vergleich mit der ursprünglichen BL4-Baseline (beam=3, len=15)

## Ergebnis (Vorschau)

| Konfiguration | WER | Δ WER | Joint Error |
|--------------|-----|-------|-------------|
| Baseline BL4 (beam=3, len=15) | 0.4987 | – | 0.3457 |
| **Final (beam=12, len=20)** | **0.4980** | **−0.0008 (−0.15%)** | **0.3453** |

Marginale Verbesserung.

**Hinweis zum Bugfix:** Dieser finale Lauf basiert auf dem Stand **vor dem Segmentierungs-Bugfix**. Der Bugfix-Lauf folgt in `03b_`. Die hier erzeugten VTT-Dateien (`output_final_bs12_len20`) dienen als Eingabe für die LLM-Postprocessing-Notebooks `03a_` und `03a2_`.

## 1 – GPU-Check

In [9]:
!nvidia-smi --query-gpu=index,memory.total,memory.used,memory.free --format=csv,noheader

0, 81920 MiB, 23041 MiB, 57996 MiB
1, 81920 MiB, 4 MiB, 81033 MiB
2, 81920 MiB, 81018 MiB, 19 MiB
3, 81920 MiB, 73976 MiB, 7061 MiB


In [10]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2"  # macht "unsere" GPU zur 0 im Prozess

import torch
print("n_gpu:", torch.cuda.device_count())
print("current device:", torch.cuda.current_device())
print("device name:", torch.cuda.get_device_name(0))


n_gpu: 1
current device: 0
device name: NVIDIA A100-SXM4-80GB


## 2 – Pfad-Konfiguration

`session = "*"` nutzt Shell-Glob – `inference.main()` verarbeitet alle Sessions
deren Verzeichnis auf dieses Muster passt.

In [11]:
project_baseline_path = "/home/josch080/Projektgruppe/mcorec_baseline" # anpassen
session = "*" # alle Sessions
session_dir = f"data-bin/dev/{session}"

In [12]:
import os, sys
os.chdir(project_baseline_path)
print("CWD:", os.getcwd())

CWD: /home/josch080/Projektgruppe/mcorec_baseline


In [13]:
from script import inference

/home/josch080/Projektgruppe/mcorec_baseline/script/inference.py:22: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


## 3 – Inference-Parameter

In [14]:
model_type = "avsr_cocktail"
checkpoint_path = "model-bin/avsr_cocktail_mcorec_finetune" # BL4-Checkpoint
beam_size = 12 # beste Kombination aus 02_ Grid-Search
max_length = 20 # beste Kombination aus 02_ Grid-Search
output_dir_name = f"output_final_bs12_len20"

## 4 – Inference starten (auskommentiert)

`sys.argv` simuliert einen Kommandozeilenaufruf für `inference.main()`.
Auskommentiert – Inference-Ergebnisse liegen bereits vor.
`session_dir` mit `*`-Glob verarbeitet alle Sessions in einem Aufruf.

In [7]:
sys.argv = [
    "notebook",
    "--model_type", model_type,
    "--session_dir", session_dir,
    "--checkpoint_path", checkpoint_path,
    "--beam_size", str(beam_size),
    "--max_length", str(max_length), 
    "--output_dir_name", output_dir_name,

    ]
#inference.main()

Loading avsr_cocktail model...


/home/josch080/Projektgruppe/mcorec_train/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading model from model-bin/avsr_cocktail_mcorec_finetune
avsr_cocktail model loaded successfully!
Inferring 25 sessions using avsr_cocktail model
Processing session session_44


Processing speakers:   0%|          | 0/6 [00:00<?, ?it/s]


Processing speakers:  17%|█▋        | 1/6 [01:21<06:47, 81.52s/it]

...


Processing speaker spk_0 track 1 of 2: 100%|██████████| 23/23 [00:41<00:00,  1.79s/it]

Processing speakers:  20%|██        | 1/5 [01:10<04:42, 70.66s/it]


Processing speakers:  40%|████      | 2/5 [02:26<03:40, 73.63s/it]


Processing speakers:  60%|██████    | 3/5 [03:24<02:13, 66.52s/it]


Processing speakers:  80%|████████  | 4/5 [03:44<00:48, 48.05s/it]


Processing speakers: 100%|██████████| 5/5 [05:38<00:00, 67.68s/it]


## 5 – Evaluation via Subprocess

`evaluate.py` wird als separater Prozess aufgerufen, damit sein Output
vollständig gepuffert und weiterverarbeitet werden kann.
Das Ergebnis wird zusätzlich in `eval_log_final_results.txt` persistiert.

In [15]:
import subprocess, sys, re, pandas as pd

eval_session_dir = session_dir   
eval_output_prefix = output_dir_name

cmd = [
    sys.executable,
    "script/evaluate.py",
    "--session_dir", eval_session_dir,
    "--output_dir_name", eval_output_prefix,
    "--label_dir_name", "labels",
]

result = subprocess.run(cmd, capture_output=True, text=True)

# Ausgabe trotzdem im Notebook sehen:
print(result.stdout)

# Ausgabe auch in eine Datei schreiben
with open("eval_log_final_results.txt", "w") as f:
    f.write(result.stdout)


Evaluating 25 sessions

=== Evaluating session session_44 ===

--- Evaluating output dir: output_final_bs12_len20 ---
Conversation clustering F1 score: 0.6
Speaker to WER: {'spk_0': 0.4006, 'spk_1': 0.6334, 'spk_2': 0.538, 'spk_3': 0.611, 'spk_4': 0.6278, 'spk_5': 0.6301}
Speaker clustering F1 score: {'spk_0': 1.0, 'spk_1': 1.0, 'spk_2': 0.5, 'spk_3': 0.5, 'spk_4': 0.5, 'spk_5': 0.5}
Joint ASR-Clustering Error Rate: {'spk_0': 0.2003, 'spk_1': 0.3167, 'spk_2': 0.519, 'spk_3': 0.5555, 'spk_4': 0.5639000000000001, 'spk_5': 0.56505}

=== Evaluating session session_57 ===

--- Evaluating output dir: output_final_bs12_len20 ---
Conversation clustering F1 score: 1.0
Speaker to WER: {'spk_0': 0.335, 'spk_1': 0.3285, 'spk_2': 0.4651, 'spk_3': 0.5205, 'spk_4': 0.5283}
Speaker clustering F1 score: {'spk_0': 1.0, 'spk_1': 1.0, 'spk_2': 1.0, 'spk_3': 1.0, 'spk_4': 1.0}
Joint ASR-Clustering Error Rate: {'spk_0': 0.1675, 'spk_1': 0.16425, 'spk_2': 0.23255, 'spk_3': 0.26025, 'spk_4': 0.26415}

=== Eva

## 6 – Ergebnis-Parsing

Regex extrahiert F1, WER und Joint Error aus dem `evaluate.py`-Output.
Das DataFrame `baseline_results` enthält eine Zeile pro Session-Experiment-Kombination.

In [16]:
# Regex: matcht einen Ergebnis-Block aus dem evaluate.py-Output
pattern = re.compile(
    r"Results for output dir prefix variant '([^']+)':\s+"
    r"\s*Average Conversation Clustering F1 score: ([0-9.]+)\s+"
    r"\s*Average Speaker WER: ([0-9.]+)\s+"
    r"\s*Average Joint ASR-Clustering Error Rate: ([0-9.]+)",
    re.MULTILINE
)

rows = []
for m in pattern.finditer(result.stdout):
    rows.append({
        "exp": m.group(1),                 # z.B. "output_avsr_cocktail_finetuned"
        "avg_conv_f1": float(m.group(2)),
        "avg_speaker_wer": float(m.group(3)),
        "avg_joint_error": float(m.group(4)),
    })

baseline_results = pd.DataFrame(rows)
# 'output_'-Prefix aus exp-Namen entfernen für lesbarere model-Spalte
baseline_results["model"] = baseline_results["exp"].str.replace("^output_", "", regex=True)
baseline_results

,exp,avg_conv_f1,avg_speaker_wer,avg_joint_error,model
0,output_final_bs12_len20,0.815282,0.497967,0.354294,final_bs12_len20


In [17]:
baseline_results.to_csv(
    "final_results.csv",
    index=False
)


## 7 – Vergleich: Final vs. BL4-Baseline

In [14]:
import pandas as pd

final_df = baseline_results.copy()   # Ergebnisse aus Abschnitt 6 (beam=12, len=20)

base_df = pd.read_csv("results_baseline_dev_without_central_videos.csv")

BASELINE_EXP = "output_avsr_cocktail_finetuned"   # BL4 Baseline: beam=3, len=15
FINAL_EXP = "output_final_bs12_len20"

def get_row(df, exp_name):
    if "exp" not in df.columns:
        raise ValueError(f"Keine Spalte 'exp' gefunden. Spalten: {list(df.columns)}")
    hit = df[df["exp"].astype(str) == exp_name]
    if hit.empty:
        print(f"[WARN] exp='{exp_name}' nicht gefunden. Verfügbare exp: {df['exp'].tolist()}")
        return df.iloc[0]
    return hit.iloc[0]

b = get_row(base_df, BASELINE_EXP)
f = get_row(final_df, FINAL_EXP)

# Runs-Tabelle (zwei Zeilen: Baseline + Final)
runs = pd.DataFrame([
    {"run": "baseline (finetuned, beam=3,len=15)", "exp": b["exp"],
     "avg_conv_f1": float(b["avg_conv_f1"]),
     "avg_speaker_wer": float(b["avg_speaker_wer"]),
     "avg_joint_error": float(b["avg_joint_error"])},
    {"run": "final (beam=12,len=20)", "exp": f["exp"],
     "avg_conv_f1": float(f["avg_conv_f1"]),
     "avg_speaker_wer": float(f["avg_speaker_wer"]),
     "avg_joint_error": float(f["avg_joint_error"])},
])

display(runs.round(6))

# Vergleichstabelle: absolute und relative Verbesserung
# F1 ↑ besser (improvement = final - baseline)
# WER/JER ↓ besser (improvement = baseline - final)
comparison = pd.DataFrame([
    {
        "metric": "Avg Conv Clustering F1 (↑)",
        "baseline": runs.loc[0, "avg_conv_f1"],
        "final": runs.loc[1, "avg_conv_f1"],
        "improvement_abs": runs.loc[1, "avg_conv_f1"] - runs.loc[0, "avg_conv_f1"],
    },
    {
        "metric": "Avg Speaker WER (↓)",
        "baseline": runs.loc[0, "avg_speaker_wer"],
        "final": runs.loc[1, "avg_speaker_wer"],
        "improvement_abs": runs.loc[0, "avg_speaker_wer"] - runs.loc[1, "avg_speaker_wer"],
    },
    {
        "metric": "Avg Joint Error (↓)",
        "baseline": runs.loc[0, "avg_joint_error"],
        "final": runs.loc[1, "avg_joint_error"],
        "improvement_abs": runs.loc[0, "avg_joint_error"] - runs.loc[1, "avg_joint_error"],
    },
])
comparison["improvement_rel_%"] = (comparison["improvement_abs"] / comparison["baseline"]) * 100

display(comparison.round(6))


,run,exp,avg_conv_f1,avg_speaker_wer,avg_joint_error
0,"baseline (finetuned, beam=3,len=15)",output_avsr_cocktail_finetuned,0.815282,0.498727,0.354674
1,"final (beam=12,len=20)",output_final_bs12_len20,0.815282,0.497967,0.354294


,metric,baseline,final,improvement_abs,improvement_rel_%
0,Avg Conv Clustering F1 (↑),0.815282,0.815282,0.00000,0.000000
1,Avg Speaker WER (↓),0.498727,0.497967,0.00076,0.152474
2,Avg Joint Error (↓),0.354674,0.354294,0.00038,0.107202


## 8 – Interpretation

Die Parameteroptimierung (beam=12, len=20) auf **allen Dev-Sessions** bestätigt
das Ergebnis vom 5-Session-Subset:

| Metrik | Baseline (beam=3, len=15) | Final (beam=12, len=20) | Δ abs | Δ rel |
|--------|--------------------------|------------------------|-------|-------|
| Speaker WER (↓) | 0.4987 | 0.4980 | −0.0008 | −0.15% |
| Joint Error (↓) | 0.3457 | 0.3453 | −0.0004 | −0.11% |
| Conv F1 (↑) | unverändert | unverändert | 0 | 0% |

Die Verbesserungen sind **marginal aber konsistent**.
Weitere Verbesserungen folgen durch den Segmentierungs-Bugfix + min_duration-Grid (`03b_`, `02k_`, `02l_`).
